In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
feature_vector_length = 784
num_classes = 10
# Load the data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Reshape the data - MLPs do not understand such things as '2D'.
# Reshape to 28 x 28 pixels = 784 features
X_train = X_train.reshape(X_train.shape[0], feature_vector_length)
X_test = X_test.reshape(X_test.shape[0], feature_vector_length)

# Convert into greyscale
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Convert target classes to categorical ones
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

input_shape = (feature_vector_length,)

In [3]:
epochs = 10
lr = 3e-4
weight_decay = 1e-4
batch_size = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [28]:
ds = tfds.load("mnist", as_supervised=True)

ds_train = (
    ds["train"]
    .shuffle(5 * batch_size)
    .batch(batch_size)
)

ds_test = (
    ds["test"]
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

ds_train = ds["train"].map(lambda self, image: tf.image.resize(image, 224, 224))

ValueError: in user code:

    <ipython-input-28-32199e27c75b>:16 None  *
        ds_train = ds["train"].map(lambda self, image: tf.image.resize(image, 224, 224))
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/ops/image_ops_impl.py:1540 resize_images_v2
        return _resize_images_common(
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/ops/image_ops_impl.py:1219 _resize_images_common
        raise ValueError('\'images\' must have either 3 or 4 dimensions.')

    ValueError: 'images' must have either 3 or 4 dimensions.


In [22]:
model = tf.keras.Sequential()
model.add(Dense(128, input_shape=input_shape, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay),
    metrics=["accuracy"],
)
model.summary()
model.fit(
    ds["train"], 
    epochs=epochs,
)


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_37 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


ValueError: in user code:

    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/

### SIMPLE TRAINING WITH SIMPLE FLATTENED IMAGE DATA

In [14]:
model = tf.keras.Sequential()
model.add(Dense(128, input_shape=input_shape, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, epochs=10, batch_size=64, verbose=1, validation_split=0.2)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
750/750 [==============================] - 1s 1ms/step - loss: 0.3366 - accuracy: 0.9054 - val_loss: 0.1803 - val_accuracy: 0.9479
Epoch 2/10
750/750 [==============================] - 1s 1ms/step - loss: 0.1540 - accuracy: 0.9555 - val_loss: 0.1336 - val_accuracy: 0.9629
Epoch 3/10
750/750 [==============================] - 1s 893us/step - loss: 0.1066 - accuracy: 0.9684 - val_loss: 0.1095 - val_accuracy: 0.9669
Epoch 4/10
750/750 [==============================] - 1s 921us/step - loss: 0.080

### SNN MNIST TRAINING